In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
from selenium import webdriver

# 1차 시도

일단 무작정 import하고 쓰기 시작했는데 chrome devtools에서는 데이터가 모두 보이는데 이렇게만 읽어오면 내가 원하는 데이터는 보이지 않는다.

In [7]:
res = requests.post('https://kbl.or.kr/game/archive-team')
print(res.content)

b'<!doctype html><html lang="ko"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no"/><title>KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80</title><meta name="author" content="KBL"/><meta name="description" content="KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80"/><meta name="referrer" content="no-referrer"/><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Expires" content="-1"/><meta http-equiv="Pragma" content="no-cache"><meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate"/><link rel="icon" href="/imgs/favicon/kbl/favicon.svg"><link rel="apple-touch-icon" type="image/png" id="apple-icon-60x60" sizes="60x60" href="/imgs/favicon/kbl/apple-icon-60x60.png"><link rel="apple-touch-icon" type="image/png" id="apple-icon-87x87" sizes="87x87" href="/imgs/favicon/kbl/apple-icon-87x87.png"><link r

In [8]:
soup = BeautifulSoup(res.content, 'html.parser')

AttributeError: module 'collections' has no attribute 'Callable'

In [97]:
soup.find_all('div', class_='archive-team-table01-wrap')

[]

# 2차 시도

검색해 본 결과, 위의 코드는 정적 페이지의 데이터를 읽어오는 것이고 동적 페이지는 webdriver를 사용해서 진행해야 내가 원하는 데이터를 가져올 수 있는 것이다.

- 그런데 아무리 사용해도 나오지 계속 같은 결과만을 가져왔다.
- 그러나~!!, ```driver.page_source```  하니까 나왔다 드디어!!!ㅜㅜ 감격스러움

- class를 더 쪼개고 해서 더 분할해서 데이터를 저장하는 것이 좋을 것 같다~

여기서 driver를 읽어오는 부분에거 경로를 정확하게 적어야 함 -> 아예 .exe로 하면 안된다.

### 팀 기록실

In [2]:
driver = webdriver.Chrome('chromedriver_win32/')
# driver.implicitly_wait(3) # 있으나 없으나
driver.get('https://kbl.or.kr/game/archive-team')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_828532\2641466854.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


#### 이거는 아님 ↓

In [182]:
from selenium.webdriver.common.by import By

In [183]:
driver.find_elements(By.CLASS_NAME, 'con-box')

[<selenium.webdriver.remote.webelement.WebElement (session="be5b8f01b60bc90fe6dc67a54977c15f", element="62ab578c-3752-420b-aeab-22f89cb7e30d")>]

이거는 아님 ↑

#### 여기부터 real

In [3]:
html = driver.page_source # page_source를 가져와야 하는 것이었다ㅠㅠ 감격스러움 진짜
soup = BeautifulSoup(html, 'html.parser')

표 헤더 가져오기

In [ ]:
header = soup.select('th') # 테이블 헤더 제목

for h in header:
    print(h.text.strip())

순위
팀
G
PTS
2P
2PA
2P%
3P
3PA
3P%
FG
FGA
FG%
FT
FTA
FT%
PP
PPA
PP%
REBOUND
AST
TO
STL
BS
PF
OFF
DEF
TOT


이건 필요 없을 듯

In [214]:
teams = soup.select('td.name') # 순위 순서로 팀 이름

for no in teams:
    print(no.text.strip())

서울 SK
울산 현대모비스
고양 캐롯
안양 KGC
전주 KCC
원주 DB
창원 LG
대구 한국가스공사
수원 KT
서울 삼성


header 밑에 데이터들 가져옴
- 맨 마지막 줄이 합계

In [219]:
rows = soup.select('tbody tr')

for r in rows:
    print(r.text.strip())

1서울 SK26
2울산 현대모비스25
2고양 캐롯25
4안양 KGC25
5전주 KCC25
6원주 DB25
7창원 LG25
8대구 한국가스공사25
9수원 KT25
10서울 삼성26
합계252
84.225.648.752.66.719.833.532.368.547.113.018.570.621.537.357.610.324.434.717.99.07.32.618.7
83.225.047.452.86.921.232.732.068.646.612.418.068.519.934.158.311.625.837.420.210.46.82.019.7
83.217.733.752.512.233.336.829.967.044.711.114.676.113.222.658.18.423.131.416.59.57.72.517.8
82.921.239.953.29.627.734.730.867.645.611.716.073.017.129.058.810.126.136.218.210.46.22.918.8
82.022.844.950.77.821.336.830.666.246.213.016.479.217.631.156.810.824.535.218.59.95.61.617.3
80.625.049.051.16.620.132.831.669.145.810.814.972.119.133.457.211.024.235.216.911.36.72.219.4
79.924.046.251.96.820.333.330.766.446.211.717.168.218.833.356.511.325.837.117.011.47.92.920.0
79.723.442.854.67.021.432.530.364.247.212.116.872.117.429.858.49.223.933.216.711.06.62.519.7
77.519.440.647.78.826.533.028.167.141.912.516.974.015.530.251.511.523.134.616.010.26.02.318.8
74.120.842.748.76.720.832.027.563.543.212.517.073.61

In [13]:
header2 = soup.select('thead tr th')

for h2 in header2:
    print(h2.text.strip())

순위
팀
G
PTS
2P
2PA
2P%
3P
3PA
3P%
FG
FGA
FG%
FT
FTA
FT%
PP
PPA
PP%
REBOUND
AST
TO
STL
BS
PF
OFF
DEF
TOT


#### html table parser 사용해보기
이거 아니야 실패야 
version dependency 때문에 갑자기 아무것도 안돼서 나 진짜 깜짝 놀랐잖아ㅠ-ㅠ

In [11]:
from html_table_parser import parser

In [4]:
data1_head = soup.select("div.archive-team-table01 table thead tr th")
print(data1_head)

[<th rowspan="2">순위</th>, <th rowspan="2">팀</th>, <th class="bd-gray" rowspan="2" style="height: 8rem;">G</th>, <th class="on" data-key="score" rowspan="2">PTS<i class="ic-align"></i></th>, <th class="" data-key="two" rowspan="2">2P<i class="ic-align"></i></th>, <th class="" data-key="twoA" rowspan="2">2PA<i class="ic-align"></i></th>, <th class="" data-key="twoR" rowspan="2">2P%<i class="ic-align"></i></th>, <th class="" data-key="three" rowspan="2">3P<i class="ic-align"></i></th>, <th class="" data-key="threeA" rowspan="2">3PA<i class="ic-align"></i></th>, <th class="" data-key="threeR" rowspan="2">3P%<i class="ic-align"></i></th>, <th class="" data-key="fg" rowspan="2">FG<i class="ic-align"></i></th>, <th class="" data-key="fgA" rowspan="2">FGA<i class="ic-align"></i></th>, <th class="" data-key="fgR" rowspan="2">FG%<i class="ic-align"></i></th>, <th class="" data-key="ft" rowspan="2">FT<i class="ic-align"></i></th>, <th class="" data-key="ftA" rowspan="2">FTA<i class="ic-align"></i

In [13]:
t = parser.HTMLTableParser(dt[0])
print(t)

In [95]:
df = pd.DataFrame(data=t[2:], columns=t[0])

In [93]:
df

,순위,팀,G
0,2,안양 KGC,27
1,3,고양 캐롯,27
2,4,울산 현대모비스,27
3,5,전주 KCC,27
4,6,원주 DB,27
5,7,창원 LG,26
6,7,대구 한국가스공사,27
7,9,수원 KT,27
8,10,서울 삼성,28
9,합계,합계,270


In [40]:
data2 = soup.find("div", {'class': 'top-scroll-table archive-team-table01'})
print(data2)

<div class="top-scroll-table archive-team-table01"><table><thead><tr><th class="on" data-key="score" row_done="False" rowspan="2">PTS<i class="ic-align"></i></th><th class="" data-key="two" row_done="False" rowspan="2">2P<i class="ic-align"></i></th><th class="" data-key="twoA" row_done="False" rowspan="2">2PA<i class="ic-align"></i></th><th class="" data-key="twoR" row_done="False" rowspan="2">2P%<i class="ic-align"></i></th><th class="" data-key="three" row_done="False" rowspan="2">3P<i class="ic-align"></i></th><th class="" data-key="threeA" row_done="False" rowspan="2">3PA<i class="ic-align"></i></th><th class="" data-key="threeR" row_done="False" rowspan="2">3P%<i class="ic-align"></i></th><th class="" data-key="fg" row_done="False" rowspan="2">FG<i class="ic-align"></i></th><th class="" data-key="fgA" row_done="False" rowspan="2">FGA<i class="ic-align"></i></th><th class="" data-key="fgR" row_done="False" rowspan="2">FG%<i class="ic-align"></i></th><th class="" data-key="ft" row_

In [11]:
records = soup.select('div.top-scroll-table.archive-team-table01 tbody tr td') # td까지 가야 요소 하나하나씩 가능해짐

for r in records:
    print(r.text.strip())

84.3
25.7
48.7
52.7
6.7
20.0
33.6
32.4
68.6
47.2
12.9
18.3
70.1
21.7
37.6
57.7
10.3
24.7
35.0
17.9
8.9
7.3
2.7
18.7
83.0
21.4
40.5
52.7
9.7
27.6
35.1
31.0
68.1
45.6
11.3
15.5
72.6
17.2
29.4
58.5
10.0
26.6
36.5
18.3
10.2
6.1
3.0
18.6
82.9
17.6
33.4
52.8
12.3
33.6
36.5
29.9
67.0
44.6
10.8
14.1
76.4
13.2
22.4
58.8
8.2
22.7
30.9
16.3
9.4
7.8
2.4
17.9
82.4
24.9
47.7
52.1
6.8
21.0
32.3
31.7
68.7
46.1
12.3
17.8
69.0
19.6
34.0
57.6
11.7
26.0
37.7
19.7
10.3
6.7
2.0
19.8
82.3
22.9
44.7
51.2
7.9
21.5
36.7
30.8
66.2
46.5
12.9
16.5
77.8
17.9
31.1
57.5
10.7
24.5
35.2
18.2
9.9
5.5
1.7
17.1
81.2
25.3
49.3
51.3
6.6
20.1
33.0
31.9
69.4
46.0
10.7
14.9
71.9
19.5
33.8
57.7
11.0
24.6
35.7
17.1
11.3
6.6
2.2
19.1
79.6
23.9
46.3
51.6
6.6
20.2
32.6
30.5
66.5
45.9
12.0
17.5
68.8
18.9
33.5
56.4
11.3
25.8
37.2
17.0
11.3
7.8
2.9
20.1
79.6
23.3
42.8
54.4
7.1
21.3
33.2
30.4
64.1
47.4
11.8
16.4
71.8
17.5
29.9
58.6
9.3
23.9
33.1
16.4
11.0
6.4
2.4
19.7
78.4
19.6
40.0
48.9
9.0
26.5
33.8
28.5
66.5
42.9
12.4
16.6
74.6
15.7

In [223]:
tdata = soup.select('tbody tr td') # 완전 요소 따로따로 가져올 때

for data in tdata:
    print(data.text.strip())

1
서울 SK
26
2
울산 현대모비스
25
2
고양 캐롯
25
4
안양 KGC
25
5
전주 KCC
25
6
원주 DB
25
7
창원 LG
25
8
대구 한국가스공사
25
9
수원 KT
25
10
서울 삼성
26
합계
252
84.2
25.6
48.7
52.6
6.7
19.8
33.5
32.3
68.5
47.1
13.0
18.5
70.6
21.5
37.3
57.6
10.3
24.4
34.7
17.9
9.0
7.3
2.6
18.7
83.2
25.0
47.4
52.8
6.9
21.2
32.7
32.0
68.6
46.6
12.4
18.0
68.5
19.9
34.1
58.3
11.6
25.8
37.4
20.2
10.4
6.8
2.0
19.7
83.2
17.7
33.7
52.5
12.2
33.3
36.8
29.9
67.0
44.7
11.1
14.6
76.1
13.2
22.6
58.1
8.4
23.1
31.4
16.5
9.5
7.7
2.5
17.8
82.9
21.2
39.9
53.2
9.6
27.7
34.7
30.8
67.6
45.6
11.7
16.0
73.0
17.1
29.0
58.8
10.1
26.1
36.2
18.2
10.4
6.2
2.9
18.8
82.0
22.8
44.9
50.7
7.8
21.3
36.8
30.6
66.2
46.2
13.0
16.4
79.2
17.6
31.1
56.8
10.8
24.5
35.2
18.5
9.9
5.6
1.6
17.3
80.6
25.0
49.0
51.1
6.6
20.1
32.8
31.6
69.1
45.8
10.8
14.9
72.1
19.1
33.4
57.2
11.0
24.2
35.2
16.9
11.3
6.7
2.2
19.4
79.9
24.0
46.2
51.9
6.8
20.3
33.3
30.7
66.4
46.2
11.7
17.1
68.2
18.8
33.3
56.5
11.3
25.8
37.1
17.0
11.4
7.9
2.9
20.0
79.7
23.4
42.8
54.6
7.0
21.4
32.5
30.3
64.2
47.2
12.1
16.

### 여기가 성공이다 테이블 가져오기 

``<table>`` 이 태그로 적힌 데이터를 가져오기 총 테이블이 2개이기 때문에 dt[0], dt[1]

In [ ]:
dt = soup.find_all('table')
table1 = pd.read_html(str(dt[0]))
table2 = pd.read_html(str(dt[1]))
table = pd.concat([table1[0], table2[0]], axis=1)

`pd.read_html`로 읽어오고 나서 리스트 안에 데이터프레임이 있기 때문에 [0]으로 접근해서 데이터프레임을 읽어와야 함

In [43]:
table1[0]
table2[0]
table

순위          팀    G   PTS    2P   2PA   2P%    3P   3PA   3P%  ...   PPA  \
    순위          팀    G   PTS    2P   2PA   2P%    3P   3PA   3P%  ...   PPA   
0    1      서울 SK   27  84.3  25.7  48.7  52.7   6.7  20.0  33.6  ...  37.6   
1    2     안양 KGC   28  83.1  21.8  40.9  53.2   9.5  27.4  34.8  ...  29.6   
2    3      고양 캐롯   27  82.9  17.6  33.4  52.8  12.3  33.6  36.5  ...  22.4   
3    4   울산 현대모비스   27  82.4  24.9  47.7  52.1   6.8  21.0  32.3  ...  34.0   
4    5     전주 KCC   27  82.3  22.9  44.7  51.2   7.9  21.5  36.7  ...  31.1   
5    6      원주 DB   28  80.8  25.1  49.0  51.2   6.6  20.0  33.2  ...  33.5   
6    7      창원 LG   26  79.6  23.9  46.3  51.6   6.6  20.2  32.6  ...  33.5   
7    7  대구 한국가스공사   27  79.6  23.3  42.8  54.4   7.1  21.3  33.2  ...  29.9   
8    9      수원 KT   27  78.4  19.6  40.0  48.9   9.0  26.5  33.8  ...  29.8   
9   10      서울 삼성   28  74.0  20.9  42.7  48.9   6.6  20.8  32.0  ...  30.3   
10  합계         합계  272  80.7  22.6  43.6  51.7   7.9  23.2  34.1  ...  31.2   

     PP% REBOUND               AST    TO  STL   BS    PF  
     PP%     OFF   DEF   TOT   AST    TO  STL   BS    PF  
0   57.7    10.3  24.7  35.0  17.9   8.9  7.3  2.7  18.7  
1   58.8     9.9  26.5  36.4  18.4  10.0  6.2  3.0  18.6  
2   58.8     8.2  22.7  30.9  16.3   9.4  7.8  2.4  17.9  
3   57.6    11.7  26.0  37.7  19.7  10.3  6.7  2.0  19.8  
4   57.5    10.7  24.5  35.2  18.2   9.9  5.5  1.7  17.1  
5   57.8    11.0  24.7  35.7  17.1  11.6  6.4  2.1  18.9  
6   56.4    11.3  25.8  37.2  17.0  11.3  7.8  2.9  20.1  
7   58.6     9.3  23.9  33.1  16.4  11.0  6.4  2.4  19.7  
8   52.8    11.1  23.3  34.5  16.6  10.4  6.1  2.2  18.9  
9   54.5     9.6  23.8  33.4  15.3  11.1  5.5  2.1  20.8  
10  57.0    10.3  24.6  34.9  17.3  10.4  6.6  2.4  19.1  

[11 rows x 27 columns]

### 선수 기록실

In [39]:
driver = webdriver.Chrome('chromedriver_win32/')
driver.get('https://kbl.or.kr/game/archive-player')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_828532\4252543053.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


In [44]:
dt = soup.find_all('table')
table1 = pd.read_html(str(dt[0]))
table2 = pd.read_html(str(dt[1]))
table = pd.concat([table1[0], table2[0]], axis=1)

In [45]:
table

순위      선수명          팀   PTS    FG   FGA   FG%   FT  FTA   FT%  ...  \
      순위      선수명          팀   PTS    FG   FGA   FG%   FT  FTA   FT%  ...   
0      1    자밀 워니      서울 SK  23.9  10.4  20.0  51.9  2.7  4.5  60.3  ...   
1      2   재로드 존스      수원 KT  20.4   8.4  16.0  52.5  0.4  0.8  50.0  ...   
2      3      전성현      고양 캐롯  20.2   6.6  14.6  45.2  2.9  3.3  88.8  ...   
3      4  오마리 스펠맨     안양 KGC  18.6   6.9  16.0  43.2  2.0  2.8  69.6  ...   
4      5   게이지 프림   울산 현대모비스  18.3   7.3  12.7  57.6  3.6  4.9  72.9  ...   
..   ...      ...        ...   ...   ...   ...   ...  ...  ...   ...  ...   
171  162      이윤수      원주 DB   0.0   0.0   0.5   0.0  0.0  0.0   0.0  ...   
172  162      임준수  대구 한국가스공사   0.0   0.0   0.0   0.0  0.0  0.0   0.0  ...   
173  162      임현택      서울 SK   0.0   0.0   0.5   0.0  0.0  0.0   0.0  ...   
174  162      정희원      서울 삼성   0.0   0.0   0.0   0.0  0.0  0.0   0.0  ...   
175  162      함준후     안양 KGC   0.0   0.0   1.0   0.0  0.0  0.0   0.0  ...   

      PPA   PP% REBOUND             AST   TO  STL   BS   PF  
      PPA   PP%     OFF  DEF   TOT  AST   TO  STL   BS   PF  
0    16.3  57.1     2.8  7.3  10.1  2.3  1.7  1.0  1.2  1.7  
1     7.6  60.5     1.4  6.0   7.4  1.8  2.8  1.0  0.2  2.2  
2     2.8  51.3     0.4  1.4   1.8  3.0  2.5  1.2  0.0  1.4  
3     5.9  53.9     2.3  7.4   9.7  2.5  1.7  0.8  1.1  2.3  
4    10.8  60.6     3.9  6.8  10.7  2.1  1.9  1.6  0.7  3.1  
..    ...   ...     ...  ...   ...  ...  ...  ...  ...  ...  
171   0.5   0.0     0.5  0.0   0.5  0.5  0.5  0.0  0.0  1.5  
172   0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  
173   0.0   0.0     0.0  0.5   0.5  0.0  0.0  0.0  0.0  0.0  
174   0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  
175   0.0   0.0     0.0  0.3   0.3  0.0  0.0  0.0  0.0  1.0  

[176 rows x 21 columns]

- 기록실 링크 정리

```https://kbl.or/game/*```

1. 일정: schedule-list
2. 팀 순위: team-rank
3. 선수 순위: player-rank-top5
4. 팀 기록실: archive-team
5. 선수 기록실: archive-player
6. 팀 기록 비교: compare-team
7. 공수력 비교: compare-offence-defence